In [13]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [14]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=False)

In [ ]:
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from controllers.directional_trading.macd_mt_dca import MacdMTDCAControllerConfig
import datetime
from decimal import Decimal

# Controller configuration
connector_name = "binance_perpetual"
trading_pair = "1000PEPE-USDT"
candles_trading_pair = ""


macd_interval_1 = "5m"
macd_signal_type_1 = 'trend_following'
macd_fast_1 = 21
macd_slow_1 = 42
macd_signal_1 = 9
macd_number_of_candles_1 = 5
macd_interval_2 = "1m"
macd_signal_type_2 = 'trend_following'
macd_fast_2 = 21
macd_slow_2 = 42
macd_signal_2 = 9
macd_number_of_candles_2 = 25
executor_refresh_time = 60*2
dca_spreads = [-0.00001,0.02]
dca_amounts  = [1,1.5]
stop_loss = 0.02
total_amount_quote = 1000
max_executors_per_side = 2
take_profit = 0.05
time_limit = 60*60*10
trailing_stop_activation_price = 0.01
trailing_stop_trailing_delta = 0.005
cooldown_time = 60*60*2

start = int(datetime.datetime(2024, 10, 20).timestamp())
end = int(datetime.datetime(2024, 10, 24).timestamp())


# Creating the instance of the configuration and the controller
config = MacdMTDCAControllerConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    executor_refresh_time = executor_refresh_time,
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    trailing_stop=TrailingStop(activation_price=Decimal(trailing_stop_activation_price), trailing_delta=Decimal(trailing_stop_trailing_delta)),
    time_limit=time_limit,
    max_executors_per_side=max_executors_per_side,
    cooldown_time=cooldown_time,
    dca_amounts = dca_amounts,
    dca_spreads = dca_spreads,
    macd_interval_1 = macd_interval_1,
    macd_signal_type_1 = macd_signal_type_1,
    macd_fast_1 = macd_fast_1,
    macd_slow_1 = macd_slow_1,
    macd_signal_1 = macd_signal_1,
    macd_number_of_candles_1 = macd_number_of_candles_1,
    macd_interval_2 = macd_interval_2,
    macd_signal_type_2 = macd_signal_type_2,    
)

In [16]:
# Running the backtesting this will output a backtesting result object that has built in methods to visualize the results

backtesting_result = await backtesting.run_backtesting(config, start, end, "1m")

2024-11-01 17:26:28,658 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f8f9a374ac0>
2024-11-01 17:26:28,661 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f8ff71fc580>, 23062.746418464)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f8f9a374b80>
2024-11-01 17:26:34,115 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f8f9a375ea0>
2024-11-01 17:26:34,118 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f8f9a39b100>, 23068.199489135)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f8f9a376230>
2024-11-01 17:26:36,104 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f8f9a375d20>
2024-11-01 17:26:36,108 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHa

In [18]:
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
backtesting_result.get_backtesting_figure()


Net PNL: $-50.12 (-5.01%) | Max Drawdown: $-54.14 (-5.41%)
Total Volume ($): 23340.02 | Sharpe Ratio: -1.92 | Profit Factor: 0.51
Total Executors: 39 | Accuracy Long: 0.59 | Accuracy Short: 0.71
Close Types: Take Profit: 0 | Stop Loss: 9 | Time Limit: 6 |
             Trailing Stop: 24 | Early Stop: 0



In [11]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
import pandas as pd

executors_df = backtesting_result.executors_df
executors_df.head()

,id,timestamp,type,close_timestamp,close_type,status,config,net_pnl_pct,net_pnl_quote,cum_fees_quote,filled_amount_quote,is_active,is_trading,custom_info,controller_id,side
0,3xv3VGmfFG9x1fihm9tZY3hMPik3KFkNyJFShrB8uWiX,1729409100,dca_executor,1729430640,CloseType.TRAILING_STOP,RunnableStatus.TERMINATED,{'id': '3xv3VGmfFG9x1fihm9tZY3hMPik3KFkNyJFShr...,0.01553308830641235774305819461460487218573689...,3.10658659510585843932517491339240223169326782...,0.11999879999999998880788609767478192225098609...,399.9959999999999809006112627685070037841796875,False,False,"{'close_price': 0.0101722, 'level_id': None, '...",None,SELL
1,EdaYJEMX4my3h9HGuz144cWkhYePWEZmUKdQtJ1yHUJ4,1729416000,dca_executor,1729434360,CloseType.TRAILING_STOP,RunnableStatus.TERMINATED,{'id': 'EdaYJEMX4my3h9HGuz144cWkhYePWEZmUKdQtJ...,0.00987827142510260339247629701731057139113545...,1.97567404156337089737860424065729603171348571...,0.12000119999999998843254189750950899906456470...,400.0040000000000190993887372314929962158203125,False,False,"{'close_price': 0.0104054, 'level_id': None, '...",None,BUY
2,6WtXUPyz5sgAqGi221VY1Qg5cVwfuJvADpSxDgydSscD,1729430700,dca_executor,1729434360,CloseType.TRAILING_STOP,RunnableStatus.TERMINATED,{'id': '6WtXUPyz5sgAqGi221VY1Qg5cVwfuJvADpSxDg...,0.02152137286104298732891670908884407253935933...,4.30431761495431963737701153149828314781188964...,0.12000119999999998843254189750950899906456470...,400.0040000000000190993887372314929962158203125,False,False,"{'close_price': 0.0104054, 'level_id': None, '...",None,BUY
3,Cr1bFmeMAD1J3GYnbnUE6fPUXMD7bBWcx5VVzdCtFnTy,1729435500,dca_executor,1729464660,CloseType.STOP_LOSS,RunnableStatus.TERMINATED,{'id': 'Cr1bFmeMAD1J3GYnbnUE6fPUXMD7bBWcx5VVzd...,-0.0199195622768400353030937566245484049431979...,-10.079258672956504128137567022349685430526733...,0.30359879999999994648618439896381460130214691...,1011.9959999999999809006112627685070037841796875,False,False,"{'close_price': 0.0107738, 'level_id': None, '...",None,SELL
4,GWoXZV2fyMQ6DRdQvooxHCwNh1djCRjxdEPJWWwdskdP,1729444500,dca_executor,1729473780,CloseType.STOP_LOSS,RunnableStatus.TERMINATED,{'id': 'GWoXZV2fyMQ6DRdQvooxHCwNh1djCRjxdEPJWW...,-0.0214618336976198469112642186473749461583793...,-10.85964492732824737686314620077610015869140625,0.30359879999999994648618439896381460130214691...,1011.9959999999999809006112627685070037841796875,False,False,"{'close_price': 0.0108575, 'level_id': None, '...",None,SELL


### Backtesting Analysis

### Scatter of PNL per Trade
This bar chart illustrates the PNL for each individual trade. Positive PNLs are shown in green and negative PNLs in red, providing a clear view of profitable vs. unprofitable trades.


In [9]:
import plotly.express as px

# Create a new column for profitability
executors_df['profitable'] = executors_df['net_pnl_quote'] > 0

# Create the scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y='net_pnl_quote',
    title='PNL per Trade',
    color='profitable',
    color_discrete_map={True: 'green', False: 'red'},
    labels={'timestamp': 'Timestamp', 'net_pnl_quote': 'Net PNL (Quote)'},
    hover_data=['filled_amount_quote', 'side']
)

# Customize the layout
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0.8)',  # Dark background
    paper_bgcolor='rgba(0,0,0,0.8)',  # Dark background for the entire plot area
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray")
)

# Add a horizontal line at y=0 to clearly separate profits and losses
fig.add_hline(y=0, line_dash="dash", line_color="lightgray")

# Show the plot
fig.show()

### Histogram of PNL Distribution
The histogram displays the distribution of PNL values across all trades. It helps in understanding the frequency and range of profit and loss outcomes.


In [10]:
fig = px.histogram(executors_df, x='net_pnl_quote', title='PNL Distribution')
fig.show()


# Conclusion
We can see that the indicator has potential to bring good signals to trade and might be interesting to see how we can design a market maker that shifts the mid price based on this indicator.
A lot of the short signals are wrong but if we zoom in into the loss signals we can see that the losses are not that big and the wins are bigger and if we had implemented the trailing stop feature probably a lot of them are going to be profits.

# Next steps
- Filter only the loss signals and understand what you can do to prevent them
- Try different configuration values for the indicator
- Test in multiple markets, pick mature markets like BTC-USDT or ETH-USDT and also volatile markets like DOGE-USDT or SHIB-USDT


from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.launch_optuna_dashboard()

In [ ]:

from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path)
optimizer.launch_optuna_dashboard()

2024-11-01 17:54:16,311 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f8fd43f7fa0>
2024-11-01 17:54:16,312 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f8fdc099ae0>
2024-11-01 17:54:16,313 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f8fd433be80>


Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

127.0.0.1 - - [01/Nov/2024 17:54:21] "GET / HTTP/1.1" 302 0
127.0.0.1 - - [01/Nov/2024 17:54:22] "GET /dashboard HTTP/1.1" 200 4145
127.0.0.1 - - [01/Nov/2024 17:54:22] "GET /static/bundle.js HTTP/1.1" 200 4159599
127.0.0.1 - - [01/Nov/2024 17:54:24] "GET /api/studies HTTP/1.1" 200 143
127.0.0.1 - - [01/Nov/2024 17:54:24] "GET /favicon.ico HTTP/1.1" 200 7670
127.0.0.1 - - [01/Nov/2024 17:54:27] "GET /api/meta HTTP/1.1" 200 60
/home/sapel/miniconda3/envs/quants-lab/lib/python3.10/site-packages/optuna_dashboard/_importance.py:75: ExperimentalWarning: PedAnovaImportanceEvaluator is experimental (supported from v3.6.0). The interface can change in the future.
  return get_param_importances(study, target=target, evaluator=PedAnovaImportanceEvaluator())
127.0.0.1 - - [01/Nov/2024 17:54:27] "GET /api/studies/1?after=0 HTTP/1.1" 200 760365
127.0.0.1 - - [01/Nov/2024 17:54:28] "GET /api/studies/1/param_importances HTTP/1.1" 200 1686
127.0